### Exemplo de uso do Multiple Dispatch, IO e DataFrames

In [ ]:
;ls

In [ ]:
include("myexp.jl")

In [ ]:
?myexp

In [ ]:
myexp(2,3)

In [ ]:
include("banktest.jl");

In [ ]:
arq = readdlm("bank-full.csv", ';', header=true)

Wow, o que é tudo isso?

In [ ]:
typeof(arq)

In [ ]:
println(size(arq[1]))
println(size(arq[2]))

In [ ]:
data = arq[1]

In [ ]:
data_names = arq[2]

In [ ]:
typeof.(data[1,:])

In [ ]:
?preprocess

In [ ]:
D = buildmultivar(data, [1, 3])

Julia também oferece a estrutura de dados de Data Frames.

In [ ]:
using DataFrames

df = DataFrame(data, Symbol.(vec(data_names)))

In [ ]:
describe(df)

In [ ]:
tail(df)

In [ ]:
?open